With our current implementation, we have a problem: [backwash](https://docs.google.com/presentation/d/1AV5v-gTSIi5xUwtm-FtkReUmuTA3Mqry1eGjje7OgQo/edit#slide=id.g11dd5164a7_2_309)

![](https://introcs.cs.princeton.edu/java/assignments/checklist/percolation-backwash.png)

If all the bottom sites are connected to each other and all the top sites are connected each other, this means sites that are connected to the bottom are all filled as well even without having to be connected to the top.

## Solution

For solution, see [here](https://github.com/alexilyenko/Algorithms1/issues/2) for more details.

One solution is to create a second `WeightedQuickUnionUF` (WQUF) object.

In [ ]:
private WeightedQuickUnionUF wquf2;

The idea is to use the main WQUF for checking if the system percolates (using the `percolates()` method), and to use `wquf2` to check if the site is filled. Some changes that need to be done:

1. For most methods that involve `union` method, `union` both the main WQUF and `wquf2`
2. Only connect all the bottom sites for the main WQUF. Don't connect `wquf2`'s bottom sites

## Changes to `Percolation(int N)` constructor

In [ ]:
public Percolation(int N) {
    super(N*N);
    /** Also call the constructor for wquf2 */
    wquf2 = new WeightedQuickUnionUF(N*N);
    openArray = new boolean[N*N];
    length = N;

    /** Top Virtual Site */
    for(int i = 1; i < length; i++) {
        union(0, i);
        /* Also union wquf2 */
        wquf2.union(0, i);
    }
    
    /* Bottom Virtual Site, only connect the
    main WQUF */
    for (int i = N*N - 2; i >= N*N - N; i--) {
        union(N*N - 1, i);
    }


## Changes to `open(int row, int col)` method

When checking adjacent sites to connect, also do the same to `wquf2`.

In [ ]:
/**
 * ABOVE
 * Checks if the site above the current site is valid (within the
 * NxN grid). If yes and the site is open, connect
 * with current site.
 */
if(row - 1 >= 0) {
    int above = convert2Dto1D(row-1, col);
    if (isOpen(row-1, col)) {
        /**
         * Connect both main wquf and second wquf
         */
        union(current, above);
        wquf2.union(current, above);
    }
}

/**
 * BELOW
 * Same with ABOVE, but for the site below current site
 */
if(row + 1 <= length - 1) {
    int below = convert2Dto1D(row+1, col);
    if (isOpen(row+1, col)) {
        union(current, below);
        wquf2.union(current, below);
    }
}

/**
 * LEFT
 */
if(col - 1 >= 0) {
    int left = convert2Dto1D(row, col - 1);
    if (isOpen(row, col-1)) {
        union(current, left);
        wquf2.union(current, left);
    }
}

/**
 * RIGHT
 */
if (col + 1 <= length-1) {
    int right = convert2Dto1D(row, col + 1);
    if (isOpen(row, col+1)) {
        union(current, right);
        wquf2.union(current,right);
    }
}


## Change to `isFull(int row, int col)`
Use the `wquf2` to determine if a site is filled.

In [ ]:
public boolean isFull(int row, int col) {
    if (isOpen(row, col)) {
        int current = convert2Dto1D(row, col);
        /** Use wquf2 */
        return wquf2.connected(current, 0);
    }
    return false;
}


## Changes (actually no changes) to `percolates()` method
Use the main system to check if the system percolates. The implementation doesn't change.

In [ ]:
public boolean percolates() {
    return connected(0, length * length - 1);
}
